In [1]:
import os
import numpy as np

import tensorflow as tf
print("TensorFlow version:", tf.__version__)

np.set_printoptions(precision=4)
seg_length = 500

TensorFlow version: 2.6.0


In [19]:
# x = None
# y = None

# for db in ["chf2db", "chfdb"]:
#     for record in os.listdir(f'data/{db}/rr'):
#         rr = np.loadtxt(f'data/{db}/rr/{record}')
#         n_segments = rr.shape[0] // seg_length
#         rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
#         labels = np.c_[np.ones(n_segments), np.zeros(n_segments)]

#         if x is None or y is None:
#             x = rr
#             y = labels
#         else:
#             x = np.r_[rr, x]
#             y = np.r_[labels, y]


# for db in ["nsrdb", "nsr2db", "fantasia"]:
#     for record in os.listdir(f'data/{db}/rr'):
#         rr = np.loadtxt(f'data/{db}/rr/{record}')
#         n_segments = rr.shape[0] // seg_length
#         rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
#         labels = np.c_[np.zeros(n_segments), np.ones(n_segments)]

#         if x is None or y is None:
#             x = rr
#             y = labels
#         else:
#             x = np.r_[rr, x]
#             y = np.r_[labels, y]

# x.shape, y.shape

X = None
y = None

db1_pos = ['chfdb']
db1_neg = ['nsrdb', 'fantasia']
db2_pos = ['chf2db']
db2_neg = ['nsr2db']

for db in db1_pos:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.ones(n_segments), np.zeros(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

for db in db1_neg:
    for record in os.listdir(f'data/{db}/rr'):
        rr = np.loadtxt(f'data/{db}/rr/{record}')
        n_segments = rr.shape[0] // seg_length
        rr = rr[:n_segments*seg_length].reshape((n_segments, seg_length))
        labels = np.c_[np.zeros(n_segments), np.ones(n_segments)]

        if X is None or y is None:
            X = rr
            y = labels
        else:
            X = np.r_[rr, X]
            y = np.r_[labels, y]

X.shape, y.shape

((7397, 500), (7397, 2))

In [20]:
# from sklearn.model_selection import train_test_split
# from sklearn import preprocessing


# x = preprocessing.scale(x, axis=1)
# # scaler = preprocessing.StandardScaler().fit_transform(x)

# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=410)
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

# X_train.shape, X_valid.shape, X_test.shape



from sklearn.model_selection import train_test_split
from sklearn import preprocessing

X = preprocessing.scale(X, axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=410)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=410)

X_train.shape, X_valid.shape, X_test.shape

((5991, 500), (666, 500), (740, 500))

In [6]:
np.unique(y_train[:,0], return_counts=True)

(array([0., 1.]), array([12796,  7996], dtype=int64))

In [5]:
inputs = tf.keras.layers.Input(shape=(500, 1))

def inception_lstm(inputs):
    a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(5, return_sequences=True))(inputs)
    a = tf.keras.layers.LSTM(5, return_sequences=True)(a)

    b = tf.keras.layers.Conv1D(5, kernel_size=1)(inputs)
    b = tf.keras.layers.Conv1D(5, kernel_size=3)(b)

    c = tf.keras.layers.Conv1D(5, kernel_size=1)(inputs)
    c = tf.keras.layers.Conv1D(5, kernel_size=5)(c)

    d = tf.keras.layers.MaxPool1D(3)(inputs)
    d = tf.keras.layers.Conv1D(5, kernel_size=1)(d)

    concat = tf.keras.layers.Concatenate(1)([a, b, c, d])
    return concat

layer1 = inception_lstm(inputs)
layer2 = inception_lstm(layer1)
dropout = tf.keras.layers.Dropout(0.2)(layer2)
flattened = tf.keras.layers.Flatten()(dropout)
output = tf.keras.layers.Dense(2, activation='sigmoid')(flattened)

model = tf.keras.Model(inputs, output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 500, 10)      280         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 500, 5)       10          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 5)       10          input_1[0][0]                    
______________________________________________________________________________________________

In [6]:
loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [8]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
          batch_size=128, epochs=100, callbacks=[callback])

Epoch 1/100
163/163 [==============================] - 126s 771ms/step - loss: 0.5902 - accuracy: 0.6903 - val_loss: 0.6900 - val_accuracy: 0.6694
Epoch 2/100
163/163 [==============================] - 126s 774ms/step - loss: 0.5874 - accuracy: 0.6944 - val_loss: 0.6617 - val_accuracy: 0.6720
Epoch 3/100
163/163 [==============================] - 128s 784ms/step - loss: 0.5836 - accuracy: 0.6949 - val_loss: 0.6417 - val_accuracy: 0.6716
Epoch 4/100
163/163 [==============================] - 129s 791ms/step - loss: 0.5826 - accuracy: 0.6971 - val_loss: 0.6472 - val_accuracy: 0.6612
Epoch 5/100
163/163 [==============================] - 158s 968ms/step - loss: 0.5851 - accuracy: 0.6943 - val_loss: 0.6581 - val_accuracy: 0.6811
Epoch 6/100
163/163 [==============================] - 154s 946ms/step - loss: 0.5813 - accuracy: 0.6955 - val_loss: 0.6796 - val_accuracy: 0.6724
Epoch 7/100
163/163 [==============================] - 154s 944ms/step - loss: 0.5846 - accuracy: 0.6966 - val_loss: 0

In [11]:
model.save('model_lstm.keras')
lstm_model = model

In [21]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

y_pred_lstm = lstm_model.predict(X_test).argmax(axis=1)

In [22]:
print(recall_score(y_test.argmax(axis=1), y_pred_lstm))
print(precision_score(y_test.argmax(axis=1), y_pred_lstm))
print(accuracy_score(y_test.argmax(axis=1), y_pred_lstm))

0.6892764857881137
0.7283276450511945
0.6575769380599922


In [21]:
from tensorflow.keras.models import load_model

lstm_model = load_model("model4 3.keras", compile=False)

In [22]:
lstm_model.summary()

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

y_pred_lstm = lstm_model.predict(X_test).argmax(axis=1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 1)]     0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 500, 10)      280         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 500, 5)       10          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 500, 5)       10          input_1[0][0]                    
______________________________________________________________________________________________

In [23]:
print(recall_score(y_test.argmax(axis=1), y_pred_lstm))
print(precision_score(y_test.argmax(axis=1), y_pred_lstm))
print(accuracy_score(y_test.argmax(axis=1), y_pred_lstm))

0.9836065573770492
0.9813084112149533
0.9797297297297297


Model 2: CNN

In [24]:
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score

def get_model():
    nclass = 2
    inp = tf.keras.layers.Input(shape=(500, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = tf.optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "baseline_cnn.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(x=X_train, y=y_train, validation_data=(X_valid, y_valid),
          batch_size=128, epochs=100, callbacks=callbacks_list)

model.load_weights(file_path)


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500, 1)]          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 496, 16)           96        
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 492, 16)           1296      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 246, 16)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 246, 16)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 244, 32)           1568      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 242, 32)           3104

In [25]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score

cnn_model = model

pred_test_cnn = cnn_model.predict(X_test)
pred_test_cnn = np.argmax(pred_test_cnn, axis=1)

print(recall_score(y_test.argmax(axis=1), pred_test_cnn ))
print(precision_score(y_test.argmax(axis=1), pred_test_cnn ))
print(accuracy_score(y_test.argmax(axis=1), pred_test_cnn ))

0.9508196721311475
0.9441860465116279
0.9391891891891891


In [26]:
import numpy as np 
import matplotlib.pyplot as plt
import stumpy
import pandas as pd
from scipy import signal
from sklearn.linear_model import Ridge
from scipy.spatial import distance
from fatf.utils.kernels import exponential_kernel 

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('C:\\Users\\Rushali\\Documents\\Code\\AI project\\listen_to_your_heart\\LIMESegment')

from Utils.explanations import LIMESegment, NEVES, LEFTIST, NNSegment, RBP, background_perturb
from  Utils.data import loadUCRDataID
from  Utils.models import *
from  Utils.metrics import *

In [58]:
cnn_exp = LIMESegment(X_test[-2], cnn_model, 'class', 'dtw', 100, 3, 50)
cnn_exp[0][0]

array([ 0.0133, -0.0345,  0.0102, -0.0277, -0.0131,  0.0302,  0.0087,
        0.0123,  0.0458,  0.0098,  0.0664, -0.0279,  0.0394,  0.0538,
        0.0469,  0.0087,  0.0517, -0.0173,  0.0168,  0.024 ,  0.0232,
        0.0436, -0.072 ,  0.0062, -0.0124, -0.0131, -0.0409,  0.0249,
       -0.0082,  0.0213,  0.0124,  0.0029,  0.0051, -0.0032,  0.0106,
       -0.0224,  0.0305,  0.0041,  0.0191, -0.0325, -0.0066,  0.0095,
       -0.0291, -0.0111,  0.0143,  0.0364,  0.0211,  0.0189,  0.0142,
        0.571 , -0.0226])

In [48]:
lstm_exp = LIMESegment(X_test[6], lstm_model, 'class', 'dtw', 100, 3, 50)
lstm_exp[0][1]

array([ 0.1946, -0.0166,  0.0066, -0.0037, -0.0002,  0.0134,  0.0068,
        0.0151, -0.007 , -0.0067,  0.002 , -0.0133,  0.0036, -0.0152,
        0.0044, -0.0018, -0.0025, -0.002 , -0.0106, -0.0028,  0.0195,
       -0.012 , -0.0042, -0.0049,  0.0038, -0.0007,  0.0163, -0.0063,
       -0.0185, -0.0187,  0.0011, -0.0081, -0.0142,  0.0136, -0.0052,
       -0.0119,  0.002 ,  0.0044,  0.0005,  0.0217,  0.0139,  0.0047,
       -0.0042, -0.0006, -0.0127,  0.0016,  0.0025,  0.0111,  0.0112,
        0.2184, -0.0113])

In [60]:
index = 112
print("Actual, CNN, LSTM")
print(y_test[index], pred_test_cnn[index], y_pred_lstm[index])
cnn_exp = LIMESegment(X_test[index], cnn_model, 'class', 'dtw', 100, 3, 5)
lstm_exp = LIMESegment(X_test[index], lstm_model, 'class', 'dtw', 100, 3, 5)
print("\nCNN")
print(cnn_exp)
print("\nLSTM")
print(lstm_exp)

Actual, CNN, LSTM
[1. 0.] 1 0

CNN
(array([[ 0.1702,  0.009 ,  0.01  ,  0.0046, -0.1612, -0.0175],
       [-0.1647, -0.0087, -0.0091, -0.0042,  0.1695,  0.0167]]), [0, 159, 160, 162, 174, 496, -1])

LSTM
(array([[-4.2554e-04,  3.0783e-03,  1.1406e-02, -1.8721e-03,  8.4891e-01,
         1.7612e-02],
       [-9.1165e-04, -3.3329e-03, -1.1539e-02,  1.4135e-03, -8.4688e-01,
        -1.7548e-02]]), [0, 159, 160, 162, 174, 496, -1])


In [71]:
count = 0
for i in range(len(y_test)):
    if y_pred_lstm[i] != pred_test_cnn[i]:
        print(i)
        count+=1
print("count mismatches", count)
print("Total samples ", len(y_test))

count = 0
for i in range(len(y_test)):
    if y_pred_lstm[i] != y_test[i][1]:
        count+=1
print("count lstm is wrong", count)

count = 0
for i in range(len(y_test)):
    if pred_test_cnn[i] != y_test[i][1]:
        count+=1
print("count cnn is wrong", count)

count = 0
for i in range(len(y_test)):
    if y_pred_lstm[i] != pred_test_cnn[i]:
        if y_pred_lstm[i] != y_test[i][1]:
            count+=1
print("count lstm is wrong in mismatched data ", count)

count = 0
for i in range(len(y_test)):
    if y_pred_lstm[i] != pred_test_cnn[i]:
        if pred_test_cnn[i] != y_test[i][1]:
            count+=1
print("count cnn is wrong in mismatched data ", count)


21
62
70
91
112
126
152
164
165
180
192
198
254
258
262
311
315
321
322
333
366
372
383
406
407
408
412
415
467
518
521
544
560
591
599
623
634
687
699
708
732
736
count mismatches 42
Total samples  740
count lstm is wrong 15
count cnn is wrong 45
count lstm is wrong in mismatched data  6
count cnn is wrong in mismatched data  36


In [65]:
index = 91
print("Actual, CNN, LSTM")
print(y_test[index], pred_test_cnn[index], y_pred_lstm[index])

Actual, CNN, LSTM
[1. 0.] 0 1


match: 
1: 28,-1 (same f, cnn>>lstm), 7,46 (same f, lstm>cnn), 
0: 6 (same f, lstm>>cnn)

mismatch:
21, 70, 112, 126 (Exact same f, opp pred) lstm got it right
62, 112 (lstm v conf, cnn not conf at all)
91, 126 (cnn conf, lstm not conf at all)

62, 91 lstm wrong 